## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 4.1

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(31,33):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.8
Dataset is using device:  cuda


2023-04-28 11:10:37.067036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:10:37.638835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 11250  MSE: 4.01e-02  Reg: 1.15e-04  L1: 1.40e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.087766)u_{xx} + (-0.993298)uu_{x}  rmse =  0.004026859081949832
Dataset is using device:  cuda
 19000  MSE: 4.22e-02  Reg: 1.09e-04  L1: 2.93e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.170427)u_{x} + (0.054825)u_{xx} + (0.135444)u + (-0.304883)uu_{x} + (0.006380)uu_{xxx} + (-0.435827)u^2 + (-0.232182)u^2u_{xx}  rmse =  0.2543805386448207
Dataset is using device:  cuda
 19500  MSE: 3.97e-02  Reg: 2.88e-04  L1: 1.70e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.145742)u_{x} + (0.014063)u_{xx} + (0.069942)u + (-0.512636)uu_{x} + (-0.261910)u^2  rmse =  0.16823554156329348
Dataset is using device:  cuda
  8500  MSE: 3.88e-02  Reg: 6.24e-05  L1: 3.65e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.197990)u_{x} + (0.036936)u_{xx} + (0.118461)u + (-0.404052)uu_{x} + (0.215944)uu_{xx} + (-0.3

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(32,33):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.75
Dataset is using device:  cuda


2023-04-29 01:03:55.620339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 01:03:56.168501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  9250  MSE: 4.35e-02  Reg: 1.17e-04  L1: 3.06e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.234768)u_{x} + (0.141468)u + (-0.357031)uu_{x} + (0.206287)uu_{xx} + (-0.354321)u^2 + (-0.220898)u^2u_{xx}  rmse =  0.24417064215971404
Dataset is using device:  cuda
 16000  MSE: 4.26e-02  Reg: 1.96e-04  L1: 2.91e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.150834)u_{x} + (0.051999)u_{xx} + (0.149989)u + (-0.439945)uu_{x} + (-0.458661)u^2 + (-0.253432)u^2u_{xx} + (0.014700)u^2u_{xxx}  rmse =  0.230222260759143
Dataset is using device:  cuda
  5250  MSE: 4.22e-02  Reg: 1.21e-04  L1: 1.65e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.175043)u_{x} + (0.053784)u_{xx} + (-0.225977)uu_{x} + (-0.103396)u^2 + (-0.490136)u^2u_{x} + (-0.053517)u^2u_{xx}  rmse =  0.27167299593359123
Dataset is using device:  cuda
 17750  MSE: 4.28e-02  Reg: 1.54e-04  L1: 2.34e+00 Algorithm converged. Writing model to disk.
Predicted PDE:

## Testing 4.2

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(33,35):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.7
Dataset is using device:  cuda


2023-04-30 12:07:08.758395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:07:09.361612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 4.72e-02  Reg: 1.81e-04  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.058978)u_{xx} + (-1.053479)uu_{x} + (0.078174)uu_{xx}  rmse =  0.029796518558175486
Dataset is using device:  cuda
 16500  MSE: 4.56e-02  Reg: 1.89e-04  L1: 1.56e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.105736)u_{xx} + (-0.942080)uu_{x} + (-0.054661)u^2u_{xx}  rmse =  0.023049546305768866
Dataset is using device:  cuda
 19750  MSE: 4.60e-02  Reg: 4.23e-04  L1: 2.76e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.140548)u_{x} + (0.019934)u_{xx} + (0.114870)u + (-0.376673)uu_{x} + (0.020281)uu_{xxx} + (-0.327740)u^2 + (-0.384841)u^2u_{x} + (-0.020580)u^2u_{xxx}  rmse =  0.2387897030280279
Dataset is using device:  cuda
 19750  MSE: 4.60e-02  Reg: 1.88e-04  L1: 3.37e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.185585)u_{x} + (0.084019)u_{xx} + (0.002020)u_{xxx} + (0.084256)u + (-0.1531

## Testing 4.3

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(35,37):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.5
Dataset is using device:  cuda


2023-05-03 10:14:02.911527: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:14:03.587491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 6.64e-02  Reg: 2.84e-04  L1: 1.74e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.078183)u_{x} + (0.043225)u_{xx} + (0.072386)u + (-0.826953)uu_{x} + (-0.204331)u^2  rmse =  0.08479014199755028
Dataset is using device:  cuda
 19750  MSE: 6.74e-02  Reg: 1.74e-04  L1: 2.70e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.059018)u_{xx} + (0.059716)u + (-0.972232)uu_{x} + (0.210272)uu_{xx} + (-0.114556)u^2 + (-0.191622)u^2u_{xx}  rmse =  0.09132044084773185
Dataset is using device:  cuda
 19750  MSE: 6.47e-02  Reg: 4.75e-04  L1: 6.30e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.024317)u_{xx} + (-0.034678)u_{xxx} + (-0.945097)uu_{x} + (0.075496)uu_{xxx} + (-0.094136)u^2 + (-0.039075)u^2u_{xxx}  rmse =  0.04657678841917906
Dataset is using device:  cuda
 19250  MSE: 6.49e-02  Reg: 2.35e-04  L1: 2.52e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.141094)u_{x} + (0.048713)u_{xx

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(36,37):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.4
Dataset is using device:  cuda


2023-05-05 10:54:01.814928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 10:54:02.441117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 12000  MSE: 8.31e-02  Reg: 1.76e-04  L1: 2.69e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.109061)u_{x} + (0.081751)u_{xx} + (-0.648436)uu_{x} + (0.131241)uu_{xx} + (-0.023283)u^2 + (-0.235524)u^2u_{xx}  rmse =  0.13199190375667877
Dataset is using device:  cuda
 16250  MSE: 8.07e-02  Reg: 2.58e-04  L1: 2.34e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.229485)u_{x} + (0.044707)u_{xx} + (0.102493)u + (-0.028023)uu_{xxx} + (-0.291539)u^2 + (-0.862671)u^2u_{x}  rmse =  0.39751388938336507
Dataset is using device:  cuda
 19750  MSE: 8.11e-02  Reg: 7.46e-04  L1: 1.38e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.306232)u_{x} + (0.062773)u + (-0.243647)u^2  rmse =  0.3118602884661254
Dataset is using device:  cuda
 15500  MSE: 7.86e-02  Reg: 4.56e-04  L1: 1.91e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.342121)u_{x} + (0.142561)u + (-0.002096)uu_{xx} + (-0.410161)u^2 + (-0.283

## Testing 4.4

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(37,39):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.3
Dataset is using device:  cuda
 19750  MSE: 1.08e-01  Reg: 6.16e-04  L1: 2.83e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.252740)u_{x} + (0.003711)u_{xxx} + (0.181003)u + (-0.015133)uu_{xxx} + (-0.467403)u^2 + (-0.664328)u^2u_{x}  rmse =  0.38369686971358075
Dataset is using device:  cuda
 19750  MSE: 1.10e-01  Reg: 6.01e-04  L1: 4.35e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.591693)u_{x} + (0.204603)u + (1.362273)uu_{x} + (-0.665425)u^2 + (-1.457529)u^2u_{x} + (-0.254195)u^2u_{xx}  rmse =  0.8472549177722453
Dataset is using device:  cuda
 19750  MSE: 1.06e-01  Reg: 7.22e-04  L1: 5.47e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.205387)u_{x} + (-0.003044)u_{xxx} + (0.149492)u + (-0.635141)uu_{x} + (0.100038)uu_{xx} + (0.008505)uu_{xxx} + (-0.525177)u^2 + (0.547003)u^2u_{x} + (-0.229602)u^2u_{xx} + (-0.018922)u^2u_{xxx}  rmse =  0.2654934356155838
Dataset is using device:  cuda
 19750 

## Testing 4.5

In [10]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(39,41):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  0.2
Dataset is using device:  cuda
 19750  MSE: 1.63e-01  Reg: 8.85e-04  L1: 8.33e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.093227)u_{xx} + (-0.013464)u_{xxx} + (-1.781338)uu_{x} + (0.527776)uu_{xx} + (0.019372)uu_{xxx} + (0.744183)u^2u_{x} + (-0.416227)u^2u_{xx}  rmse =  0.3712572677034435
Dataset is using device:  cuda
  5250  MSE: 1.57e-01  Reg: 5.37e-04  L1: 1.30e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.227280)u_{x} + (0.009621)u_{xx} + (-0.053227)u + (-0.272828)uu_{x} + (-0.488975)u^2u_{x}  rmse =  0.26308012009920484
Dataset is using device:  cuda
 19750  MSE: 1.65e-01  Reg: 1.09e-03  L1: 3.73e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.037783)u_{x} + (0.027922)u_{xx} + (0.001661)u_{xxx} + (0.128101)u + (-1.042294)uu_{x} + (-0.417950)u^2 + (1.248572)u^2u_{x} + (-0.129373)u^2u_{xx}  rmse =  0.38461855424401803
Dataset is using device:  cuda
 19750  MSE: 1.65e-01  Reg: 9.75e-04  L1